In [1]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
import networkx as nx
from scipy import linalg
from scipy import sparse as sp
from sklearn.cluster import KMeans
%matplotlib inline

## Data
Sample graph of friends in social network

In [2]:
data = np.load('network.npy').astype(float)

In [4]:
data

array([[0., 0., 1., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [5]:
adj = sp.csr_matrix(data)
adj

<128x128 sparse matrix of type '<class 'numpy.float64'>'
	with 2048 stored elements in Compressed Sparse Row format>

# Implementing spectral clustering (to predict links in graph)

**Based on https://link.springer.com/content/pdf/10.1007%2Fs10618-010-0210-x.pdf**

## 1. Construct the graph Laplacian

First, we need to construct the Laplacian for the given graph. 

Given the **adjacency matrix** $A \in \mathbb{R}^{N \times N},$ we define the **degree matrix** $D \in \mathbb{R}^{N \times N}$ of an undirected graph as
$$D_{ij} =  \begin{cases}\sum_{k=1}^N A_{ik} & if \;\; i = j\\ 0 & if \;\; i \ne j\end{cases}$$

Our goal is to minimze the **normalized cut**, we will need to use the **normalized Laplacian** (a.k.a. symmetrized Laplacian), defined as
$$L_{sym} = I - D^{-1/2}AD^{-1/2}$$

In [6]:
from scipy.sparse.csgraph import laplacian
import scipy

def construct_laplacian(A, norm_laplacian):
    """Construct Laplacian of a graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    norm_laplacian : bool
        Whether to construct the normalized graph Laplacian or not.
        If True, construct the normalized (symmetrized) Laplacian, L = I - D^{-1/2} A D^{-1/2}.
        If False, construct the unnormalized Laplacian, L = D - A.
        
    Returns
    -------
    L : scipy.sparse.csr_matrix, shape [N, N]
        Laplacian of the graph.
        
    """
    
    if norm_laplacian:
        D_diag = sp.csr_matrix(sp.diags([np.sqrt(1/np.sum(row)) for row in A]))
        
        return scipy.sparse.csr_matrix(sp.eye(*A.shape) -  D_diag.dot(A.dot(D_diag)))
    else:
        return sp.diags([np.sum(row) for row in A]) - A

In [7]:
from functools import partial
# get_L = partial(construct_laplacian, norm_laplacian=False)
get_L_sym = partial(construct_laplacian, norm_laplacian=True)

## 2. Spectral embedding

Now, we have to compute the spectral embedding for the given graph.

In order to partition the graph into $k$ clusters, such that the desired cut (ratio or normalized) is minimized, we need to consider the $k$ eigenvectors corresponding to the $k$ smallest eigenvalues of the graph Laplacian.

Since the Laplacian matrix is sparse and symmetric, we can use the function `eigsh` from the `scipy.sparse.linalg` package in order to find eigendecomposition of $L$ (`eig` - eigendecomposition, `s` - sparse, `h`- Hermitian).
The function `eigsh` directly allows you to find the smallest / largest eigenvalues by specifying the `k` and `which` parameters. 

Keep in mind that the Laplacian matrix is always positive semi-definite when picking the appropriate value for the `which` parameter.

In [8]:
from scipy.sparse.linalg import eigsh

In [9]:
def spectral_embedding(A, num_clusters, get_L_func=get_L_sym):
    """Compute spectral embedding of nodes in the given graph.
    
    Parameters
    ----------
    A : scipy.sparse.csr_matrix, shape [N, N]
        Adjacency matrix of the graph.
    num_clusters : int
        Number of clusters to detect in the data.
    get_L_func : callable, default get_L
        Function to calcualte Lapassian for A matrix.
    Returns
    -------
    embedding : np.array, shape [N, num_clusters]
        Spectral embedding for the given graph.
        Each row represents the spectral embedding of a given node.
    
    """
    
    L_ = get_L_func(A)
    eigvals, eigenvecs = eigsh(L_, k=num_clusters+1, which='SA')
    
    return eigenvecs[...,1:]

In [10]:
spectral_embedding(adj, 10, get_L_sym).shape

(128, 10)

## 3. Generate positive and negative samples for link prediction

In [33]:
import random

In [15]:
positive = []
negative = []
adj_dense = adj.todense()
for i in range(adj_dense.shape[0]):
    for j in range(i, adj_dense.shape[1]):
        if adj_dense[i,j] == 1:
            positive.append((i,j))
        else:
            negative.append((i,j))

In [35]:
positive = np.array(positive)

In [36]:
negative = np.array(negative)

In [37]:
negative = negative[random.sample(range(0, len(negative)), len(positive))]

In [38]:
positive.shape

(1024, 2)

In [39]:
negative.shape

(1024, 2)

## 4. Generate features for edges

According to this paper https://arxiv.org/pdf/1607.00653.pdf it's better to choose L1 norm (hence tests on more complicated datasets showed that for spectral clustering L1 is the best)

In [40]:
k = 10
embed = spectral_embedding(adj, k, get_L_sym)

def wL1(a,b):
    return np.abs(a-b)

In [41]:
features = []
for elem in positive:
    features.append(wL1(embed[elem[0]], embed[elem[1]]))

pos_features = np.array(features)

In [42]:
features = []
for elem in negative:
    features.append(wL1(embed[elem[0]], embed[elem[1]]))

neg_features = np.array(features)

In [43]:
features = np.concatenate((pos_features, neg_features), axis=0)
features.shape

(2048, 10)

In [47]:
labels = np.concatenate((np.ones(pos_features.shape[0]), np.zeros(neg_features.shape[0])), axis=0)

In [48]:
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from time import time

In [49]:
X = features
y = labels
# build a classifier
clf = svm.SVC(gamma='auto')


# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


skf = StratifiedKFold(n_splits=5, shuffle=True)

# use a full grid over all parameters
param_grid = {"C": [1, 2, 3],
              "kernel": ['linear', 'poly', 'rbf', 'sigmoid']
             }

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid, cv=skf.split(X,y), scoring='roc_auc')
start = time()
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

GridSearchCV took 8.47 seconds for 12 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.887 (std: 0.023)
Parameters: {'C': 1, 'kernel': 'poly'}

Model with rank: 1
Mean validation score: 0.887 (std: 0.023)
Parameters: {'C': 2, 'kernel': 'poly'}

Model with rank: 1
Mean validation score: 0.887 (std: 0.023)
Parameters: {'C': 3, 'kernel': 'poly'}

